# SIT720 Assignment 3
Bryon Baker

Student number: 85031775

In [1]:
!pip3 install numpy
!pip3 install pandas
!pip3 install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import warnings
import random
import pandas as pd # dataframe manipulation
import numpy as np # linear algebra

# Configure ssl for unverified content so we can load a dataset from an unknown source.
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
url = 'https://raw.githubusercontent.com/bryonbaker/datasets/main/SIT720/Ass3/ac_train_data.csv'
train_df = pd.read_csv(url)
print(f"{train_df.head()}\n")

   Unnamed: 0   load  ac  hourofday dayofweek    dif  absdif    max       var  \
0      105541  2.245   0          0       Sun  0.987   0.987  6.215  3.074549   
1      105542  2.259   0          0       Sun  0.014   0.014  6.215  3.172867   
2      105543  2.269   0          0       Sun  0.010   0.010  6.215  3.270112   
3      105544  2.268   0          0       Sun -0.001   0.001  6.215  3.303763   
4      105545  2.270   0          0       Sun  0.002   0.002  6.215  3.302744   

    entropy  nonlinear     hurst  
0  0.678886   0.052903  0.994071  
1  0.667450   0.054829  0.994154  
2  0.647777   0.056991  0.994220  
3  0.629227   0.057606  0.994150  
4  0.621295   0.082640  0.994041  



In [5]:
url = 'https://raw.githubusercontent.com/bryonbaker/datasets/main/SIT720/Ass3/ac_test_data.csv'
test_df = pd.read_csv(url)
print(f"{test_df.head()}\n")

   Unnamed: 0   load  ac  hourofday dayofweek    dif  absdif  max  var  \
0           1  1.869   0          0       Mon  0.000   0.000  0.0  0.0   
1           2  1.673   0          0       Mon -0.196   0.196  0.0  0.0   
2           3  1.660   0          0       Mon -0.013   0.013  0.0  0.0   
3           4  1.772   0          0       Mon  0.112   0.112  0.0  0.0   
4           5  1.679   0          0       Mon -0.093   0.093  0.0  0.0   

   entropy  nonlinear  hurst  
0      0.0        0.0    0.0  
1      0.0        0.0    0.0  
2      0.0        0.0    0.0  
3      0.0        0.0    0.0  
4      0.0        0.0    0.0  

